### Word Embeddings: Word2Vec -Skip Gram

En este cuaderno de trabajo ilustramos la obtención de representaciones distribuidas de palabras (*word embeddings*) a partir de un corpus en español.

**Preparación**

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
np.set_printoptions(precision=5, suppress=True)

print(tf.__version__)

# Corrección del dtype para evitar la advertencia de futuro
_np_qint8 = np.dtype([("qint8", np.int8, (1,))])


Trabajaremos con el texto de "Don Quijote de la Mancha".

In [ ]:
texto = open('Quijote.txt', 'rb').read().decode(encoding='utf-8')
texto = texto[708:-19255]  # Eliminamos el encabezado y pie del texto (en inglés, no es parte de la obra)
print ('Tamaño del texto: {} caracteres'.format(len(texto)))
print ()
print (texto[:200])

Utilizamos la librería [`tf.keras.preprocessing.text.Tokenizer`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) para extraer un vocabulario del texto, asignar un índice a cada palabra del vocabulario, y representar el texto como una secuencia de valores enteros:

In [ ]:
# Independientemente del número de palabras, el tokenizer limitará el número de
# índices asignados a sólo 2000 palabras diferentes.
TAM_VOCAB = 1500

tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=TAM_VOCAB,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\r\n\'¡¿«»',
    oov_token='<OOV>'
)
tokenizer.fit_on_texts([texto])

print ('El tokenizador encontró %d palabras en el texto' % len(tokenizer.word_index))
print ('pero sólo considerará %d en el vocabulario.\n' % TAM_VOCAB)


# El Tokenizer convierte entre secuencias de enteros y palabras.
# Las siguientes funciones nos facilitarán el trabajo
def sequence_to_text(sequence):
  return tokenizer.sequences_to_texts([sequence])[0]

def idx_to_word(idx):
  return sequence_to_text([idx])

def text_to_sequence(text):
  return tokenizer.texts_to_sequences([text])[0]

def word_to_idx(word):
  return text_to_sequence([word])[0]


# Convertimos el texto en una secuencia de números enteros
sequence = text_to_sequence(texto)


# Veamos las primeras palabras de la secuencia
print('Inicio de la secuencia:\n')
print('Índice  Palabra')
print('------  -------')
for idx in sequence[:15]:
  print('{:6d}  {}'.format(idx, idx_to_word(idx)))

In [ ]:
# Veamos las primeras palabras de la secuencia
print('Inicio de la secuencia:\n')
print('Índice  Palabra')
print('------  -------')
for idx in range(10):
  print('{:6d}  {}'.format(idx, idx_to_word(idx)))

**Word2Vec: Skip Gram**

Word2Vec es una familia de modelos para la creación de representaciones distribuidas de palabras. Se inspiran en la idea de que el significado de una palabra puede ser capturado a partir de su contexto.

El embedding Skip Gram de una palabra es obteniendo al aprender a predecir las palabras que se encuentran en su contexto.

Definamos y probemos primero una función que nos permita transformar una secuencia en un generador de pares *(x, y)* que relacionen una palabra con las que se encuentran a *skip_window* o menos posiciones de distancia. Para simplificar, y dado que lo usaremos en secuencias largas, omitiremos el problema de lidiar con las palabras de los extremos.


In [ ]:
# Función para armar los pares x, y
def genera_muestras(sequence, skip_window):
  window_size = 2 * skip_window + 1
  for i in range(len(sequence) - window_size + 1):
    window = sequence[i : i + window_size]
    x = window[skip_window]
    for j in range(window_size):
      if j != skip_window:
        yield [[x], [window[j]]]

def prueba_generacion_muestras():
  for sample in genera_muestras([0, 1, 2, 3, 4, 5, 6, 7], 2):
    print (sample)

prueba_generacion_muestras()

In [ ]:
# Esta función encapsula generate_samples para poder enviarla como argumento a tf.data.Dataset.from_generator
def generador():
  return genera_muestras(sequence, skip_window=3)

# tf.data.Dataset importa los valores como un tensor de dos elementos para cada ejemplo
# Con esta función los convertiremos en un tuple (input, label), que es lo que esperará Keras
def division_muestras(sample):
  return sample[0], sample[1]

TAM_BATCH = 128
TAM_BUFFER = 20000

# Creación del dataset usando TensorFlow 1.13.1 API
dataset = tf.data.Dataset.from_generator(generador, output_types=tf.int32, output_shapes=(2,1))
dataset = dataset.map(division_muestras)
dataset = dataset.shuffle(TAM_BUFFER).batch(TAM_BATCH, drop_remainder=True)
dataset

### Construcción del modelo

In [ ]:
DIM_EMBEDDING = 128

# Usamos el API Funcional de Keras

entradas = tf.keras.Input(shape=(1,))
entradas_embedded= tf.keras.layers.Embedding(TAM_VOCAB, DIM_EMBEDDING, input_length=1)(entradas)
logits = tf.keras.layers.Dense(TAM_VOCAB)(entradas_embedded)
modelo = tf.keras.Model(inputs=entradas, outputs=logits)

modelo.summary()

### Entrenamiento del modelo

**Uso de callbacks**

Preparemos una función *Callback* que nos permita ir visualizando la calidad de los embeddings que obtendremos. Cada vez que se ejecute, imprimirá las 8 palabras más parecidas a 15 palabras aleatorias de entre las más frecuentes del vocabulario.

In [ ]:
val_words = ['quijote', 'él', 'dijo', 'tres', 'duque', 'mal', 'eres', 'boca', 'mundo', 'quiero', 'padre', 'hombre', 'había']
val_indices = [word_to_idx(x) for x in val_words]

# Imprimir muestra de palabras con las que les son más similares
def palabras_mas_similares(batch):
  if batch % 500 != 0:
    return

  # Obtener los embeddings
  embeddings = modelo.layers[1].get_weights()[0]  # (TAM_VOCAB, DIM_EMBEDDING)

  # Normalizar los embeddings
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
  normalized_embeddings = embeddings / norm
  val_embeddings = np.take(normalized_embeddings, val_indices, axis=0)

  # Calcular la matriz de similaridad de coseno (producto punto de vectores normalizados)
  similaridad = tf.matmul(val_embeddings, tf.transpose(normalized_embeddings))

  # Buscamos e imprimimos las palabras más cercanas a las palabras aleatorias que elegimos
  print()
  for i, (val_word, val_idx) in enumerate(zip(val_words, val_indices)):
    top_k = 8 # número de palabras más cercanas
    nearest = tf.argsort(-similaridad[i, :])[1:top_k+1].numpy()
    print('Más similares a %-10s: %s' % (val_word, ', '.join(sequence_to_text(nearest).split())))
  print()

visualization_callback = tf.keras.callbacks.LambdaCallback(on_batch_end=lambda batch,logs: palabras_mas_similares(batch))

# Checkpoint CallBack
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

**Asignamiento de un optimizador y una función de pérdida**

In [ ]:
def perdida(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

modelo.compile(optimizer='adam', loss=perdida)

**Ejecutación del entrenamiento**

In [ ]:
EPOCAS=1  # Aprox. 10 minutos por época en Colab
history = modelo.fit(dataset, epochs=EPOCAS, callbacks=[checkpoint_callback, visualization_callback])

**Recuperar los embeddings aprendidos**

In [ ]:
embeddings = modelo.layers[1].get_weights()[0]  # (TAM_VOCAB, DIM_EMBEDDING)

print('Dimensiones de la matriz de embeddings : ', embeddings.shape)

ejemplo = 'quijote'
ejemplo_idx = word_to_idx(ejemplo)
print(f'''
Ejemplo
-------
Palabra   :  {ejemplo}
Ìndice    :  {ejemplo_idx}
Embedding :
{embeddings[ejemplo_idx]}
''')

**Visualizar en el proyector de embeddings de TensorFlow**

Para esta visualización necesitamos grabar y descargar los vectores de embeddings y la lista de palabras como archivos de texto separados por tabs (.tsv):



In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for i in range(TAM_VOCAB):
  out_m.write(idx_to_word(i) + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings[i]]) + "\n")
out_v.close()
out_m.close()

**Ejercicio**

Carga ambos archivos en el [Proyector de embeddings de TensorFlow](http://projector.tensorflow.org), usando la opción Load. Analiza los resultados.

In [ ]:
# Tu respuesta

El modelo CBOW predice una palabra objetivo basándose en las palabras de contexto circundantes. Experimenta con el código siguiente de acuerdo a lo visto a clase.

In [ ]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

class CBOW:
    def __init__(self, vocab_size, embedding_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.W1 = np.random.rand(vocab_size, embedding_dim)
        self.W2 = np.random.rand(embedding_dim, vocab_size)

    def train(self, context, target, epochs=1000, learning_rate=0.01):
        for epoch in range(epochs):
            h = np.mean(self.W1[context], axis=0)
            u = np.dot(h, self.W2)
            y_pred = softmax(u)

            # Error
            EI = np.array(y_pred)
            EI[target] -= 1

            # Backpropagacion
            dW2 = np.outer(h, EI)
            dW1 = np.dot(self.W2, EI).reshape(self.W1[context].shape)

            self.W1[context] -= learning_rate * dW1
            self.W2 -= learning_rate * dW2

            if epoch % 100 == 0:
                print(f'Epoca {epoch}, Perdida: {np.sum(-np.log(y_pred[target]))}')

    def word_vector(self, word_idx):
        return self.W1[word_idx]

# Ejemplo
vocab_size = 10  
embedding_dim = 5
modelo = CBOW(vocab_size, embedding_dim)
contextos = [1, 2, 3, 4]  
objetivo = 5  
modelo.train(contextos, objetivo)


In [ ]:
## Tu respuesta

El modelo Skip-Gram funciona de manera opuesta al CBOW, intenta predecir las palabras de contexto a partir de la palabra objetivo. Experimenta con el código siguiente de acuerdo a lo visto a clase.

In [ ]:
class SkipGram:
    def __init__(self, vocab_size, embedding_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.W1 = np.random.rand(vocab_size, embedding_dim)
        self.W2 = np.random.rand(embedding_dim, vocab_size)

    def train(self, target, contexts, epochs=1000, learning_rate=0.01):
        for epoch in range(epochs):
            h = self.W1[target]
            u = np.dot(h, self.W2)
            y_pred = softmax(u)

            EI = np.array(y_pred)
            EI[contexts] -= 1 / len(contexts)

            dW2 = np.outer(h, EI)
            dW1 = np.dot(self.W2, EI).reshape(self.W1[target].shape)

            self.W1[target] -= learning_rate * dW1
            self.W2 -= learning_rate * dW2

            if epoch % 100 == 0:
                print(f'Epoca {epoch}, Perdida: {np.sum(-np.log(y_pred[contexts]))}')

    def word_vector(self, word_idx):
        return self.W1[word_idx]

# Ejemplo
vocab_size = 10
embedding_dim = 5
modelo = SkipGram(vocab_size, embedding_dim)
objetivo = 5  
contextos = [1, 2, 3, 4]  # indices of context words
modelo.train(objetivo, contextos)


In [ ]:
## Tu respuesta

Puedes modificar el código de este cuaderno en TensorFlow 2.10.1  e implementar el modelo CBOW visto en clases y analizar la visualización de tus resultados.

In [ ]:
# Tu respuesta